In [64]:
import numpy as np
import pandas as pd
import difflib
from scipy.sparse import find

from sklearn.feature_extraction.text import TfidfVectorizer 
# for transforming the textual data to numerical feature vector to find present similarity values 
from sklearn.metrics.pairwise import cosine_similarity

In [65]:
dataset=pd.read_csv(r"C:\Users\LENOVO\Downloads\movies.csv")
dataset

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

In [66]:
dataset.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [67]:
dataset.shape

(4803, 24)

In [68]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [69]:
# checking for null values in dataset

dataset.isnull().sum()
     

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [70]:
# Total count of null values

dataset.isnull().sum().sum()

4454

In [71]:
# changing the title of movies to lowercase 

dataset['original_title'] = dataset['original_title'].str.lower()
print(dataset['original_title'].head())

0                                      avatar
1    pirates of the caribbean: at world's end
2                                     spectre
3                       the dark knight rises
4                                 john carter
Name: original_title, dtype: object


In [72]:
selected_features = ['genres', 'keywords', 'tagline', 'cast','runtime', "director"]
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'runtime', 'director']


In [73]:
# replacing the null values with null string

for features in selected_features :
  dataset[features] = dataset[features].fillna('')

#### Combining the selected features

In [74]:
dataset['runtime'] = dataset['runtime'].astype(str)

In [75]:
combined_features = dataset['genres']+''+dataset['keywords']+''+dataset['tagline']+''+dataset['cast']+''+dataset['runtime']

In [76]:
combined_features.head()

0    Action Adventure Fantasy Science Fictioncultur...
1    Adventure Fantasy Actionocean drug abuse exoti...
2    Action Adventure Crimespy based on novel secre...
3    Action Crime Drama Thrillerdc comics crime fig...
4    Action Adventure Science Fictionbased on novel...
dtype: object

In [77]:
print(combined_features )

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object


In [78]:
vectorizer = TfidfVectorizer()
feature_vector = vectorizer.fit_transform(combined_features )

In [79]:
print(feature_vector)
     

  (0, 19821)	0.27679007119607085
  (0, 15872)	0.15864130465063278
  (0, 13938)	0.24217224329423276
  (0, 22227)	0.16343694559322805
  (0, 25361)	0.19848940803966114
  (0, 21467)	0.20423445234782203
  (0, 20558)	0.21780863953484125
  (0, 26290)	0.19983048297821354
  (0, 25960)	0.2373148775426801
  (0, 20587)	0.15827275653240075
  (0, 17726)	0.26401366092151857
  (0, 17350)	0.08783883829379473
  (0, 25939)	0.12755006679538788
  (0, 23114)	0.07599044433032053
  (0, 21765)	0.27679007119607085
  (0, 4477)	0.24791728760239362
  (0, 25153)	0.13199552465670022
  (0, 21894)	0.3402924504599892
  (0, 9819)	0.16709443831387827
  (0, 4266)	0.22033082566694695
  (0, 8967)	0.27679007119607085
  (0, 20870)	0.10073304498551933
  (0, 8611)	0.11932538937292772
  (0, 460)	0.09206888338322607
  (0, 266)	0.07942722957163033
  :	:
  (4801, 5234)	0.33188071652182954
  (4801, 21189)	0.33188071652182954
  (4801, 26132)	0.33188071652182954
  (4801, 26270)	0.31656136572946825
  (4801, 7818)	0.2750534117813915
  (

#### Cosine Similiarity

In [80]:
# similarity confidence value using cosine similiarity

similarity = cosine_similarity(feature_vector)

In [81]:
print(similarity)

[[1.         0.07084182 0.01538787 ... 0.         0.         0.        ]
 [0.07084182 1.         0.02915734 ... 0.01287857 0.         0.        ]
 [0.01538787 0.02915734 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.01287857 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [82]:
print(similarity.shape)
# compare with all the other movies present there

(4803, 4803)


In [83]:
# creating a list with all the moviee names in dataset

list_of_all_titles = dataset['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

### Recommedation

In [84]:
# getting the name as input for recommendation

movie_name = input('Enter your favourite movie name : ')

Enter your favourite movie name : Iron man


In [85]:
# finding the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [86]:
close_match = find_close_match[0]
print(close_match)

Iron Man


In [87]:
# Index of movie with title and data

index_of_movie = dataset[dataset.title == close_match]['index'].values[0]
# we are finding the index of movie by comparing the title of each movie with closly matched string and returing its index 
print(index_of_movie)

68


In [88]:
# finding similariy score 

score = list(enumerate(similarity[index_of_movie]))
print(score)

[(0, 0.01557018546092943), (1, 0.03540854218734534), (2, 0.00621239645032504), (3, 0.006655374324847993), (4, 0.015436373645409233), (5, 0.006215388714714171), (6, 0.023026107660171578), (7, 0.15287018708512487), (8, 0.0), (9, 0.03711099721107494), (10, 0.018244236790057468), (11, 0.0055871033986788984), (12, 0.0), (13, 0.005862772153433781), (14, 0.04734189202360697), (15, 0.0), (16, 0.19522522864692754), (17, 0.005885795006362263), (18, 0.01533680275514847), (19, 0.0052066108990419845), (20, 0.07064970562716226), (21, 0.005283051911443764), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.16851164041292296), (27, 0.014198713848642328), (28, 0.06259187520294415), (29, 0.006373301027739461), (30, 0.07322492689955441), (31, 0.22905195558917843), (32, 0.020667454000232493), (33, 0.13191994397218124), (34, 0.0), (35, 0.033217071934125084), (36, 0.030562659737113296), (37, 0.0), (38, 0.10612063949719482), (39, 0.07922969798664502), (40, 0.0), (41, 0.014758776928727034), (42, 0.0), (43, 0

In [89]:
len(score)

4803

In [90]:
# finding movies having higher similarity score 

sorted_list = sorted(score, key = lambda x:x[1], reverse = True)
# sort based on non increasing order, key is based on first index of score in list element which is score in (index, score)
print(sorted_list)

[(68, 1.0), (31, 0.22905195558917843), (79, 0.2047498925127373), (16, 0.19522522864692754), (26, 0.16851164041292296), (7, 0.15287018708512487), (33, 0.13191994397218124), (46, 0.12540611465210924), (511, 0.11932917047006329), (353, 0.11866440550564589), (203, 0.11685474315215674), (174, 0.11294236113588203), (64, 0.1078202700445429), (38, 0.10612063949719482), (85, 0.10588429364070195), (954, 0.10582999928490325), (1210, 0.10548370409664683), (2235, 0.10505629576987635), (101, 0.1045195697279622), (3166, 0.10022711582431411), (788, 0.09818166420827139), (1406, 0.09566175880975615), (882, 0.09476701546937685), (2063, 0.09088509587668309), (2186, 0.09061008076086075), (1282, 0.09028557540545855), (182, 0.08793125112572324), (307, 0.08732710027546736), (3443, 0.0861969502853917), (356, 0.08465395493295287), (205, 0.08372535160131381), (2411, 0.08288832262501042), (2066, 0.08262496281862625), (2447, 0.08141181446599696), (783, 0.0813303099506407), (4033, 0.08066911973695451), (1664, 0.080

In [91]:
print(sorted_list[0], sorted_list[1], sorted_list[2], sorted_list[3], sorted_list[4])
# top five score with index 

(68, 1.0) (31, 0.22905195558917843) (79, 0.2047498925127373) (16, 0.19522522864692754) (26, 0.16851164041292296)


In [92]:
# printing the name of simila rmovies based on the index

print("Movies Results : \n")

i = 1

for movie in sorted_list:
  index = movie[0]
  title_of_movie = dataset[dataset.index == index]['title'].values[0]
  if(i<=5):
    print(i, '.', title_of_movie)
    i += 1

Movies Results : 

1 . Iron Man
2 . Iron Man 3
3 . Iron Man 2
4 . The Avengers
5 . Captain America: Civil War


### Recommendation function

In [97]:
def recommend(name):
  find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
  
  try:
    close_match = find_close_match[0]
  except IndexError:
    close_match = 'null'
    print("No result found with this title")
    return 
  index_of_movie = dataset[dataset.title == close_match]['index'].values[0]
  score = list(enumerate(similarity[index_of_movie]))
  sorted_list = sorted(score, key = lambda x:x[1], reverse = True)


  print("Movies Results : \n")

  i = 1

  for movie in sorted_list:
    index = movie[0]
    title_of_movie = dataset[dataset.index == index]['title'].values[0]
    if(i<=5):
      print(i, '. ', title_of_movie)
      i += 1

In [98]:
movie_name = input('Enter your favourite movie name : ')

recommend(movie_name)

Enter your favourite movie name : Avatar
Movies Results : 

1 .  Avatar
2 .  Alien
3 .  Guardians of the Galaxy
4 .  Moonraker
5 .  Space Dogs
